## get the data 

In [30]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split

Soil_types_directory= 'C:/Users/user/Google Drive/TCD20/python/dataset/Soil types'
data_image=pd.read_csv(os.path.join(Soil_types_directory,'all_image_32.csv'))
data_image_mean=pd.read_csv(os.path.join(Soil_types_directory,'all_image_mean.csv'))
data_image_select_image=pd.read_csv(os.path.join(Soil_types_directory,'Selecting_image_64.csv'))
data_image_cleaning=pd.read_csv(os.path.join(Soil_types_directory,'all_image_cleaning_128.csv'))


## help function

In [4]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import  confusion_matrix,multilabel_confusion_matrix
from sklearn.metrics import classification_report,precision_recall_fscore_support
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import  MinMaxScaler
from sklearn.preprocessing import PolynomialFeatures,QuantileTransformer,PowerTransformer
from sklearn.preprocessing import StandardScaler,RobustScaler

#from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn import set_config



def def_GridSearchCV(pipeline_object,param_grid,X,Y):
    # run GridSearchCV
    grid_search = GridSearchCV(pipeline_object,param_grid , scoring='accuracy', cv=5, n_jobs=-1, verbose=2)
    grid_search.fit(X,Y) 
    #cols =['param_'+param for param in list(grid_search.param_grid[0].keys())] +['mean_test_score','std_test_score','rank_test_score']
    cv_results_df = pd.DataFrame(grid_search.cv_results_).sort_values(by = 'mean_test_score', ascending=False)#[cols]
    filter_col = [col for col in cv_results_df if col.startswith('param')]+['mean_test_score','std_test_score']
    cv_results_df= cv_results_df[filter_col]
    best_params=grid_search.best_params_
    best_score=grid_search.best_score_
    param_grid_cv=grid_search.param_grid
    return    cv_results_df, param_grid_cv,best_params,best_score, grid_search

def  Summary_results_CVׁׂ(cv_results_df):
     cv_results_df['str_model']=cv_results_df['param_classifier'].apply(str)
     base_pramter=cv_results_df.groupby(by=['str_model']).apply(lambda df: df.query('mean_test_score==mean_test_score.max()'))
     filter_col = [col for col in cv_results_df if col.startswith('param')]+['mean_test_score','std_test_score']
     base_pramter=  base_pramter[filter_col].reset_index(drop=True).sort_values(by = 'mean_test_score', ascending=False)#
     return   base_pramter 
     
 

def base_model_predict(grid_search,X,Y):
    # predict train
    predict_train= grid_search.predict(X)
    predict_df=Y.copy().assign( predict_class=predict_train )
    
    
    classification_report1=classification_report(predict_df['classes'],
                                                      predict_df['predict_class'],
                                                      labels=grid_search.classes_,
                                                      output_dict=True)
  
    multi_confusion_matrix1=multilabel_confusion_matrix(predict_df['classes'],
                                       predict_df['predict_class'],
                                       labels=grid_search.classes_)
    
      
    confusion_matrix1= confusion_matrix(predict_df['classes'],
                                       predict_df['predict_class'],
                                       labels=grid_search.classes_)
    
    return  predict_df,classification_report1,  confusion_matrix1,multi_confusion_matrix1
 
    
def base_model_summary(grid_search,classification_report,multi_confusion_matrix,confusion_matrix):
    
    # classification_report to DataFrame
    required_fields=['Black Soil', 'Cinder Soil', 'Laterite Soil', 'Peat Soil', 'Yellow Soil','macro avg', 'weighted avg']#, 'weighted avg'
    classification_report_dict={k:v['precision'] for k,v in classification_report.items() if k in required_fields}
    classification_report_dict.update({'accuracy':classification_report1['accuracy']})
    classification_report_df=pd.DataFrame(classification_report_dict,index=['precision']).T
    classification_report_df= classification_report_df.sort_values(by = 'precision', ascending=False)#
    
    # confusion_matrix to  DataFrame
    dict_matrix=dict()
    classes=grid_search.classes_
    for i,l in zip(range(multi_confusion_matrix.shape[0]),classes):
        cm=multi_confusion_matrix1[i,:,:]
        df_cm = pd.DataFrame(cm,index= ['other',l], columns=['other',l])
        #confusion_matrix, index=class_names, columns=class_names,
        dict_matrix.update({l:df_cm})

    confusion_matrix_df= pd.DataFrame(confusion_matrix,index=classes, columns=classes)
    return classification_report_df, dict_matrix, confusion_matrix_df
    

# Building the model process (Pipeline)
##  examine preprocessing need

PolynomialFeatures 2 3 4
QuantileTransformer
PowerTransformer

In [27]:
def  examine_preprocessing_need_SVC(c,gamma):
     full_pipeline_SVC = Pipeline(steps=[
  
     ('preprocessor',None),
     ('norm',MinMaxScaler()), 
     ('classifier', SVC(C=c,gamma=gamma))
     ])

     param_grid_processes= [
     {},
    
     {
     'preprocessor':[PolynomialFeatures()],   
     'preprocessor__degree':[2,3,4] 
      },
      {'preprocessor':[QuantileTransformer()]
      },
      {'preprocessor':[PowerTransformer()],
      'preprocessor__method':['yeo-johnson']   
      }]
     return    full_pipeline_SVC,  param_grid_processes
    



############################################################################################################3
full_pipeline_SVC = Pipeline(steps=[
  
   ('preprocessor',None),
   ('norm',MinMaxScaler()), 
   ('classifier', SVC(C=10,gamma=0.0005))
   ])

full_pipeline_RF = Pipeline(steps=[
    
   ('preprocessor',None),
   ( 'norm',MinMaxScaler()),
   ('classifier',RandomForestClassifier(n_estimators=500,max_depth=50,min_samples_split=10))
   ])


full_pipeline_KNN = Pipeline(steps=[
   ('preprocessor',None),
   ( 'norm',MinMaxScaler()),
   ('classifier',KNeighborsClassifier(n_neighbors=11))
   ])



param_grid_processes= [
    {},
    
    {
     'preprocessor':[PolynomialFeatures()],   
     'preprocessor__degree':[2,3,4] 
      },
    {'preprocessor':[QuantileTransformer()]
    },
    {'preprocessor':[PowerTransformer()],
   'preprocessor__method':['yeo-johnson']   
}]
    
  

## examine the best model and parameters
LogisticRegression
KNeighbors
SVC
RandomForest


In [20]:
pipeline = Pipeline(steps=[
    ('norm',MinMaxScaler()),
    ('preprocessor',None),
    ('classifier', None)
])


param_grid= [
    {
   
     'classifier': [ LogisticRegression(solver='liblinear')],
     'classifier__penalty': ['l1', 'l2'],
     'classifier__C': [0.1,0.5,1,5,10],
     },
   {  
        'classifier': [KNeighborsClassifier()],
        'classifier__n_neighbors': [2,3,7,11,15,21,31],
    },
     {
       
        'classifier': [SVC()],
        'classifier__C': [0.1,0.5,1,5,10,20,50],
         'classifier__gamma':[0.00001,0.00005,0.0001,0.0005,0.001,0.005,0.01,0.05],
         'preprocessor':[PolynomialFeatures(degree=3)]
    },
    {
         
        'classifier': [RandomForestClassifier()],
        'classifier__n_estimators': [50,100,200,500],
         'classifier__max_depth':[1, 10, 50, 100],
         'classifier__min_samples_split':[2,5,10,15],
         'preprocessor':[PolynomialFeatures(degree=2)]
    },
]




#  data image all pixel

###  train_test_split data

In [11]:
data_image2=data_image.groupby(by=['classes']).sample(frac=0.01, random_state=42)
print(data_image2.shape)

x_data=data_image2[['r','g','b']]
y_data=data_image2[['classes']]
X_train, X_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=42)


(1597, 10)


### necessary processes


In [12]:
# SVC
print('################## SVC  ################')
cv_results_df, param_grid_cv,best_params,best_score, grid_search=def_GridSearchCV(full_pipeline_SVC ,param_grid_processes,X_train,y_train)
print(best_params,best_score)
cv_results_df

################## SVC  ################
Fitting 5 folds for each of 6 candidates, totalling 30 fits


C:\Users\user\anaconda3\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


{'preprocessor': PolynomialFeatures(degree=3), 'preprocessor__degree': 3} 0.3633670343137255


,param_preprocessor,param_preprocessor__degree,param_preprocessor__method,params,mean_test_score,std_test_score
2,PolynomialFeatures(degree=3),3,NaN,"{'preprocessor': PolynomialFeatures(degree=3),...",0.363367,0.010256
3,PolynomialFeatures(degree=3),4,NaN,"{'preprocessor': PolynomialFeatures(degree=3),...",0.361026,0.014465
1,PolynomialFeatures(degree=3),2,NaN,"{'preprocessor': PolynomialFeatures(degree=3),...",0.317142,0.015096
4,QuantileTransformer(),NaN,NaN,{'preprocessor': QuantileTransformer()},0.281143,0.013051
5,PowerTransformer(),NaN,yeo-johnson,"{'preprocessor': PowerTransformer(), 'preproce...",0.234145,0.001872
0,NaN,NaN,NaN,{},0.231795,0.003054


In [13]:
#RF
print('################## RF  ################')
cv_results_df, param_grid_cv,best_params,best_score, grid_search=def_GridSearchCV(full_pipeline_RF ,param_grid_processes,X_train,y_train)
print(best_params,best_score)
cv_results_df

################## RF  ################
Fitting 5 folds for each of 6 candidates, totalling 30 fits


C:\Users\user\anaconda3\lib\site-packages\sklearn\pipeline.py:405: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)


{'preprocessor': PolynomialFeatures(), 'preprocessor__degree': 2} 0.7063388480392156


,param_preprocessor,param_preprocessor__degree,param_preprocessor__method,params,mean_test_score,std_test_score
1,PolynomialFeatures(),2,NaN,"{'preprocessor': PolynomialFeatures(), 'prepro...",0.706339,0.024215
3,PolynomialFeatures(),4,NaN,"{'preprocessor': PolynomialFeatures(), 'prepro...",0.706333,0.022309
2,PolynomialFeatures(),3,NaN,"{'preprocessor': PolynomialFeatures(), 'prepro...",0.706330,0.018087
4,QuantileTransformer(),NaN,NaN,{'preprocessor': QuantileTransformer()},0.702414,0.029157
5,PowerTransformer(),NaN,yeo-johnson,"{'preprocessor': PowerTransformer(), 'preproce...",0.702414,0.025070
0,NaN,NaN,NaN,{},0.697708,0.026396


In [14]:
# KNN
print('################## KNN  ################')  

cv_results_df, param_grid_cv,best_params,best_score, grid_search=def_GridSearchCV(full_pipeline_KNN ,param_grid_processes,X_train,y_train)
print(best_params,best_score)
cv_results_df

################## KNN  ################
Fitting 5 folds for each of 6 candidates, totalling 30 fits
{} 0.6953860294117649


C:\Users\user\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:952: UserWarning: One or more of the test scores are non-finite: [0.69538603 0.68677696        nan        nan 0.67971814 0.68835784]
  warnings.warn(
C:\Users\user\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


,param_preprocessor,param_preprocessor__degree,param_preprocessor__method,params,mean_test_score,std_test_score
0,NaN,NaN,NaN,{},0.695386,0.018658
5,PowerTransformer(),NaN,yeo-johnson,"{'preprocessor': PowerTransformer(), 'preproce...",0.688358,0.024775
1,PolynomialFeatures(),2,NaN,"{'preprocessor': PolynomialFeatures(), 'prepro...",0.686777,0.016584
4,QuantileTransformer(),NaN,NaN,{'preprocessor': QuantileTransformer()},0.679718,0.009033
2,PolynomialFeatures(),3,NaN,"{'preprocessor': PolynomialFeatures(), 'prepro...",NaN,NaN
3,PolynomialFeatures(),4,NaN,"{'preprocessor': PolynomialFeatures(), 'prepro...",NaN,NaN


### best model and parameters no  preprocessing

In [ ]:
cv_results_df, param_grid_cv,best_params,best_score, grid_search=def_GridSearchCV(pipeline ,param_grid,X_train,y_train)
print(best_params,best_score)
base_pramter_per_model=Summary_results_CVׁׂ(cv_results_df)
base_pramter_per_model


Fitting 5 folds for each of 137 candidates, totalling 685 fits


C:\Users\user\anaconda3\lib\site-packages\sklearn\pipeline.py:405: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)


{'classifier': RandomForestClassifier(max_depth=50, min_samples_split=5, n_estimators=500), 'classifier__max_depth': 50, 'classifier__min_samples_split': 5, 'classifier__n_estimators': 500, 'preprocessor': PolynomialFeatures()} 0.7047610294117648


,param_classifier,param_classifier__C,param_classifier__penalty,param_classifier__n_neighbors,param_classifier__gamma,param_preprocessor,param_classifier__max_depth,param_classifier__min_samples_split,param_classifier__n_estimators,params,mean_test_score,std_test_score
2,"RandomForestClassifier(max_depth=50, min_sampl...",NaN,NaN,NaN,NaN,PolynomialFeatures(),50,5,500,{'classifier': RandomForestClassifier(max_dept...,0.704761,0.022734
0,KNeighborsClassifier(),NaN,NaN,11,NaN,NaN,NaN,NaN,NaN,"{'classifier': KNeighborsClassifier(), 'classi...",0.695377,0.013950
3,SVC(),50,NaN,NaN,0.05,PolynomialFeatures(degree=3),NaN,NaN,NaN,"{'classifier': SVC(), 'classifier__C': 50, 'cl...",0.684424,0.014370
1,LogisticRegression(solver='liblinear'),5,l1,NaN,NaN,NaN,NaN,NaN,NaN,{'classifier': LogisticRegression(solver='libl...,0.646823,0.027536


## predict and summary base  model 

In [28]:
Soil_types_directory= 'C:/Users/user/Google Drive/TCD20/python/dataset/Soil types'

predict_df,classification_report1,confusion_matrix1,multi_confusion_matrix1=base_model_predict(grid_search,X_test, y_test)
classification_report_df, dict_matrix, confusion_matrix_df= base_model_summary(grid_search,classification_report1,multi_confusion_matrix1,confusion_matrix1)
classification_report_df.to_csv(os.path.join(Soil_types_directory,'ml_all_image.csv'))
#classification_report_df
print('### Yellow Soil ###')
print(dict_matrix['Yellow Soil'])
print('### Laterite Soil	 ###')
print(dict_matrix['Laterite Soil'])
print('### Black Soil ###')
print(dict_matrix['Black Soil'])
print('### Cinder Soill	 ###')
print(dict_matrix['Cinder Soil'])
print('### Peat Soil	 ###')
print(dict_matrix['Peat Soil'])


print(confusion_matrix_df)
classification_report_df


### Yellow Soil ###
             other  Yellow Soil
other          263            5
Yellow Soil     11           41
### Laterite Soil	 ###
               other  Laterite Soil
other            247             17
Laterite Soil      4             52
### Black Soil ###
            other  Black Soil
other         217          23
Black Soil     16          64
### Cinder Soill	 ###
             other  Cinder Soil
other          233           22
Cinder Soil     36           29
### Peat Soil	 ###
           other  Peat Soil
other        229         24
Peat Soil     24         43
               Black Soil  Cinder Soil  Laterite Soil  Peat Soil  Yellow Soil
Black Soil             64            4              1          9            2
Cinder Soil            18           29              6         12            0
Laterite Soil           0            0             52          1            3
Peat Soil               5           14              5         43            0
Yellow Soil             0        

,precision
Yellow Soil,0.891304
Laterite Soil,0.753623
Black Soil,0.735632
macro avg,0.718196
accuracy,0.715625
weighted avg,0.710507
Peat Soil,0.641791
Cinder Soil,0.568627


# data image Selecting

## train_test_split data

In [31]:
data_image_Selecting2=data_image_select_image.groupby(by=['classes']).sample(frac=0.05, random_state=42)
print(data_image_Selecting2.shape)
x_data=data_image_Selecting2[['r','g','b']]
y_data=data_image_Selecting2[['classes']]
X_train, X_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=42)


(1025, 10)


### necessary processes


In [32]:
# SVC
print('################## SVC  ################')
cv_results_df, param_grid_cv,best_params,best_score, grid_search=def_GridSearchCV(full_pipeline_SVC ,param_grid_processes,X_train,y_train)
print(best_params,best_score)
cv_results_df

##############################################################################
print('################################################################')


#############################################################################


################## SVC  ################
Fitting 5 folds for each of 6 candidates, totalling 30 fits
{'preprocessor': PolynomialFeatures(degree=3), 'preprocessor__degree': 3} 0.39146341463414636


C:\Users\user\anaconda3\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


,param_preprocessor,param_preprocessor__degree,param_preprocessor__method,params,mean_test_score,std_test_score
2,PolynomialFeatures(degree=3),3,NaN,"{'preprocessor': PolynomialFeatures(degree=3),...",0.391463,2.160981e-02
3,PolynomialFeatures(degree=3),4,NaN,"{'preprocessor': PolynomialFeatures(degree=3),...",0.387805,2.743225e-02
1,PolynomialFeatures(degree=3),2,NaN,"{'preprocessor': PolynomialFeatures(degree=3),...",0.368293,1.368899e-02
0,NaN,NaN,NaN,{},0.207317,2.775558e-17
4,QuantileTransformer(),NaN,NaN,{'preprocessor': QuantileTransformer()},0.207317,2.775558e-17
5,PowerTransformer(),NaN,yeo-johnson,"{'preprocessor': PowerTransformer(), 'preproce...",0.207317,2.775558e-17


In [33]:
# #RF
print('################## RF  ################')
cv_results_df, param_grid_cv,best_params,best_score, grid_search=def_GridSearchCV(full_pipeline_RF ,param_grid_processes,X_train,y_train)
print(best_params,best_score)
cv_results_df

################## RF  ################
Fitting 5 folds for each of 6 candidates, totalling 30 fits


C:\Users\user\anaconda3\lib\site-packages\sklearn\pipeline.py:405: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)


{'preprocessor': PolynomialFeatures(degree=3), 'preprocessor__degree': 3} 0.8524390243902438


,param_preprocessor,param_preprocessor__degree,param_preprocessor__method,params,mean_test_score,std_test_score
2,PolynomialFeatures(degree=3),3,NaN,"{'preprocessor': PolynomialFeatures(degree=3),...",0.852439,0.027378
0,NaN,NaN,NaN,{},0.850000,0.027160
1,PolynomialFeatures(degree=3),2,NaN,"{'preprocessor': PolynomialFeatures(degree=3),...",0.847561,0.023458
3,PolynomialFeatures(degree=3),4,NaN,"{'preprocessor': PolynomialFeatures(degree=3),...",0.847561,0.027540
4,QuantileTransformer(),NaN,NaN,{'preprocessor': QuantileTransformer()},0.846341,0.024813
5,PowerTransformer(),NaN,yeo-johnson,"{'preprocessor': PowerTransformer(), 'preproce...",0.845122,0.029772


In [34]:
# SVC
print('##################  KNN  ################')
cv_results_df, param_grid_cv,best_params,best_score, grid_search=def_GridSearchCV(full_pipeline_KNN ,param_grid_processes,X_train,y_train)
print(best_params,best_score)
cv_results_df

##################  KNN  ################
Fitting 5 folds for each of 6 candidates, totalling 30 fits
{} 0.8390243902439025


C:\Users\user\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:952: UserWarning: One or more of the test scores are non-finite: [0.83902439 0.83536585        nan        nan 0.82804878 0.83658537]
  warnings.warn(
C:\Users\user\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


,param_preprocessor,param_preprocessor__degree,param_preprocessor__method,params,mean_test_score,std_test_score
0,NaN,NaN,NaN,{},0.839024,0.020981
5,PowerTransformer(),NaN,yeo-johnson,"{'preprocessor': PowerTransformer(), 'preproce...",0.836585,0.023267
1,PolynomialFeatures(degree=3),2,NaN,"{'preprocessor': PolynomialFeatures(degree=3),...",0.835366,0.023773
4,QuantileTransformer(),NaN,NaN,{'preprocessor': QuantileTransformer()},0.828049,0.019049
2,PolynomialFeatures(degree=3),3,NaN,"{'preprocessor': PolynomialFeatures(degree=3),...",NaN,NaN
3,PolynomialFeatures(degree=3),4,NaN,"{'preprocessor': PolynomialFeatures(degree=3),...",NaN,NaN


### best model and parameters

In [35]:
cv_results_df, param_grid_cv,best_params,best_score, grid_search=def_GridSearchCV(pipeline ,param_grid,X_train,y_train)
print(best_params,best_score)
base_pramter_per_model=Summary_results_CVׁׂ(cv_results_df)
base_pramter_per_model

Fitting 5 folds for each of 137 candidates, totalling 685 fits
{'classifier': KNeighborsClassifier(n_neighbors=3), 'classifier__n_neighbors': 3} 0.8658536585365854


C:\Users\user\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


,param_classifier,param_classifier__C,param_classifier__penalty,param_classifier__n_neighbors,param_classifier__gamma,param_preprocessor,param_classifier__max_depth,param_classifier__min_samples_split,param_classifier__n_estimators,params,mean_test_score,std_test_score
0,KNeighborsClassifier(n_neighbors=3),NaN,NaN,3,NaN,NaN,NaN,NaN,NaN,{'classifier': KNeighborsClassifier(n_neighbor...,0.865854,0.023458
2,"RandomForestClassifier(max_depth=50, min_sampl...",NaN,NaN,NaN,NaN,PolynomialFeatures(),100,5,200,{'classifier': RandomForestClassifier(max_dept...,0.853659,0.029622
3,SVC(),50,NaN,NaN,0.05,PolynomialFeatures(degree=3),NaN,NaN,NaN,"{'classifier': SVC(), 'classifier__C': 50, 'cl...",0.820732,0.035888
1,LogisticRegression(solver='liblinear'),5,l1,NaN,NaN,NaN,NaN,NaN,NaN,{'classifier': LogisticRegression(solver='libl...,0.790244,0.028234


## predict and summary base  model 

In [36]:
Soil_types_directory= 'C:/Users/user/Google Drive/TCD20/python/dataset/Soil types'


predict_df,classification_report1,confusion_matrix1,multi_confusion_matrix1=base_model_predict(grid_search,X_test, y_test)
classification_report_df, dict_matrix, confusion_matrix_df= base_model_summary(grid_search,classification_report1,multi_confusion_matrix1,confusion_matrix1)
classification_report_df.to_csv(os.path.join(Soil_types_directory,'ml_Selecting_image.csv'))

#classification_report_df
print('### Yellow Soil ###')
print(dict_matrix['Yellow Soil'])
print('### Laterite Soil	 ###')
print(dict_matrix['Laterite Soil'])
print('### Black Soil ###')
print(dict_matrix['Black Soil'])
print('### Cinder Soill	 ###')
print(dict_matrix['Cinder Soil'])
print('### Peat Soil	 ###')
print(dict_matrix['Peat Soil'])


print(confusion_matrix_df)
classification_report_df


### Yellow Soil ###
             other  Yellow Soil
other          162            0
Yellow Soil      0           43
### Laterite Soil	 ###
               other  Laterite Soil
other            159              1
Laterite Soil      1             44
### Black Soil ###
            other  Black Soil
other         165           5
Black Soil      0          35
### Cinder Soill	 ###
             other  Cinder Soil
other          153            8
Cinder Soil     17           27
### Peat Soil	 ###
           other  Peat Soil
other        152         15
Peat Soil     11         27
               Black Soil  Cinder Soil  Laterite Soil  Peat Soil  Yellow Soil
Black Soil             35            0              0          0            0
Cinder Soil             1           27              1         15            0
Laterite Soil           0            1             44          0            0
Peat Soil               4            7              0         27            0
Yellow Soil             0        

,precision
Yellow Soil,1.000000
Laterite Soil,0.977778
Black Soil,0.875000
accuracy,0.858537
weighted avg,0.858519
macro avg,0.853413
Cinder Soil,0.771429
Peat Soil,0.642857


# data image mean
 

## train_test_split data

In [37]:
x_data=data_image_mean[['r','g','b']]
print(x_data.shape)
y_data=data_image_mean[['classes']]
X_train, X_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=42)



(156, 3)


### necessary processes

In [38]:
# SVC
print('################## SVC  ################')
cv_results_df, param_grid_cv,best_params,best_score, grid_search=def_GridSearchCV(full_pipeline_SVC ,param_grid_processes,X_train,y_train)
print(best_params,best_score)
cv_results_df

################## SVC  ################
Fitting 5 folds for each of 6 candidates, totalling 30 fits
{'preprocessor': QuantileTransformer()} 0.24166666666666664


C:\Users\user\anaconda3\lib\site-packages\sklearn\preprocessing\_data.py:2627: UserWarning: n_quantiles (1000) is greater than the total number of samples (124). n_quantiles is set to n_samples.
  warnings.warn(
C:\Users\user\anaconda3\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


,param_preprocessor,param_preprocessor__degree,param_preprocessor__method,params,mean_test_score,std_test_score
4,QuantileTransformer(),NaN,NaN,{'preprocessor': QuantileTransformer()},0.241667,0.042400
5,PowerTransformer(),NaN,yeo-johnson,"{'preprocessor': PowerTransformer(), 'preproce...",0.241667,0.042400
0,NaN,NaN,NaN,{},0.233667,0.028292
1,PolynomialFeatures(degree=3),2,NaN,"{'preprocessor': PolynomialFeatures(degree=3),...",0.233667,0.028292
2,PolynomialFeatures(degree=3),3,NaN,"{'preprocessor': PolynomialFeatures(degree=3),...",0.233667,0.012667
3,PolynomialFeatures(degree=3),4,NaN,"{'preprocessor': PolynomialFeatures(degree=3),...",0.233667,0.012667


In [39]:
# #RF
print('################## RF  ################')
cv_results_df, param_grid_cv,best_params,best_score, grid_search=def_GridSearchCV(full_pipeline_RF ,param_grid_processes,X_train,y_train)
print(best_params,best_score)
cv_results_df

################## RF  ################
Fitting 5 folds for each of 6 candidates, totalling 30 fits


C:\Users\user\anaconda3\lib\site-packages\sklearn\pipeline.py:405: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self._final_estimator.fit(Xt, y, **fit_params_last_step)


{'preprocessor': PolynomialFeatures(degree=3), 'preprocessor__degree': 3} 0.6856666666666668


,param_preprocessor,param_preprocessor__degree,param_preprocessor__method,params,mean_test_score,std_test_score
2,PolynomialFeatures(degree=3),3,NaN,"{'preprocessor': PolynomialFeatures(degree=3),...",0.685667,0.027721
3,PolynomialFeatures(degree=3),4,NaN,"{'preprocessor': PolynomialFeatures(degree=3),...",0.678000,0.047497
1,PolynomialFeatures(degree=3),2,NaN,"{'preprocessor': PolynomialFeatures(degree=3),...",0.677667,0.033393
4,QuantileTransformer(),NaN,NaN,{'preprocessor': QuantileTransformer()},0.677667,0.033393
0,NaN,NaN,NaN,{},0.661333,0.018086
5,PowerTransformer(),NaN,yeo-johnson,"{'preprocessor': PowerTransformer(), 'preproce...",0.661333,0.040089


In [40]:
# KNN
print('################## KNN  ################')  

cv_results_df, param_grid_cv,best_params,best_score, grid_search=def_GridSearchCV(full_pipeline_KNN ,param_grid_processes,X_train,y_train)
print(best_params,best_score)
cv_results_df

################## KNN  ################
Fitting 5 folds for each of 6 candidates, totalling 30 fits
{'preprocessor': PowerTransformer(), 'preprocessor__method': 'yeo-johnson'} 0.7023333333333333


C:\Users\user\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:952: UserWarning: One or more of the test scores are non-finite: [0.67833333 0.67833333        nan        nan 0.66166667 0.70233333]
  warnings.warn(
C:\Users\user\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


,param_preprocessor,param_preprocessor__degree,param_preprocessor__method,params,mean_test_score,std_test_score
5,PowerTransformer(),NaN,yeo-johnson,"{'preprocessor': PowerTransformer(), 'preproce...",0.702333,0.098362
1,PolynomialFeatures(degree=3),2,NaN,"{'preprocessor': PolynomialFeatures(degree=3),...",0.678333,0.077531
0,NaN,NaN,NaN,{},0.678333,0.092580
4,QuantileTransformer(),NaN,NaN,{'preprocessor': QuantileTransformer()},0.661667,0.080898
2,PolynomialFeatures(degree=3),3,NaN,"{'preprocessor': PolynomialFeatures(degree=3),...",NaN,NaN
3,PolynomialFeatures(degree=3),4,NaN,"{'preprocessor': PolynomialFeatures(degree=3),...",NaN,NaN


## best model and parameters

In [41]:
cv_results_df, param_grid_cv,best_params,best_score, grid_search=def_GridSearchCV(pipeline ,param_grid,X_train,y_train)
print(best_params,best_score)
base_pramter_per_model=Summary_results_CVׁׂ(cv_results_df)
base_pramter_per_model

Fitting 5 folds for each of 137 candidates, totalling 685 fits
{'classifier': KNeighborsClassifier(n_neighbors=3), 'classifier__n_neighbors': 3} 0.7416666666666667


C:\Users\user\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


,param_classifier,param_classifier__C,param_classifier__penalty,param_classifier__n_neighbors,param_classifier__gamma,param_preprocessor,param_classifier__max_depth,param_classifier__min_samples_split,param_classifier__n_estimators,params,mean_test_score,std_test_score
0,KNeighborsClassifier(n_neighbors=3),NaN,NaN,3,NaN,NaN,NaN,NaN,NaN,{'classifier': KNeighborsClassifier(n_neighbor...,0.741667,0.042400
1,LogisticRegression(solver='liblinear'),10,l1,NaN,NaN,NaN,NaN,NaN,NaN,{'classifier': LogisticRegression(solver='libl...,0.725000,0.092195
3,SVC(),50,NaN,NaN,0.05,PolynomialFeatures(degree=3),NaN,NaN,NaN,"{'classifier': SVC(), 'classifier__C': 50, 'cl...",0.693333,0.074952
2,"RandomForestClassifier(max_depth=50, min_sampl...",NaN,NaN,NaN,NaN,PolynomialFeatures(),10,5,50,{'classifier': RandomForestClassifier(max_dept...,0.685667,0.027721


## predict and summary base model

In [44]:
Soil_types_directory= 'C:/Users/user/Google Drive/TCD20/python/dataset/Soil types'



predict_df,classification_report1,confusion_matrix1,multi_confusion_matrix1=base_model_predict(grid_search,X_test, y_test)
classification_report_df, dict_matrix, confusion_matrix_df= base_model_summary(grid_search,classification_report1,multi_confusion_matrix1,confusion_matrix1)
classification_report_df.to_csv(os.path.join(Soil_types_directory,'ml_mea_image.csv'))
#classification_report_df
print('### Yellow Soil ###')
print(dict_matrix['Yellow Soil'])
print('### Laterite Soil	 ###')
print(dict_matrix['Laterite Soil'])
print('### Black Soil ###')
print(dict_matrix['Black Soil'])
print('### Cinder Soill	 ###')
print(dict_matrix['Cinder Soil'])
print('### Peat Soil	 ###')
print(dict_matrix['Peat Soil'])


print(confusion_matrix_df)
classification_report_df


### Yellow Soil ###
             other  Yellow Soil
other           27            1
Yellow Soil      0            4
### Laterite Soil	 ###
               other  Laterite Soil
other             23              0
Laterite Soil      1              8
### Black Soil ###
            other  Black Soil
other          22           1
Black Soil      0           9
### Cinder Soill	 ###
             other  Cinder Soil
other           25            4
Cinder Soil      2            1
### Peat Soil	 ###
           other  Peat Soil
other         23          2
Peat Soil      5          2
               Black Soil  Cinder Soil  Laterite Soil  Peat Soil  Yellow Soil
Black Soil              9            0              0          0            0
Cinder Soil             0            1              0          2            0
Laterite Soil           0            0              8          0            1
Peat Soil               1            4              0          2            0
Yellow Soil             0        

,precision
Laterite Soil,1.0000
Black Soil,0.9000
Yellow Soil,0.8000
weighted avg,0.7625
accuracy,0.7500
macro avg,0.6800
Peat Soil,0.5000
Cinder Soil,0.2000


# data cleaning data

## train_test_split data

In [45]:
data_image2=data_image_cleaning.groupby(by=['classes']).sample(frac=0.01, random_state=42)
print(data_image2.shape)

x_data=data_image2[['r','g','b']]
y_data=data_image2[['classes']]
X_train, X_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=42)


(3066, 11)


## best model and parameters

In [ ]:
cv_results_df, param_grid_cv,best_params,best_score, grid_search=def_GridSearchCV(pipeline ,param_grid,X_train,y_train)
print(best_params,best_score)
base_pramter_per_model=Summary_results_CVׁׂ(cv_results_df)
base_pramter_per_model

## predict and summary base model

In [ ]:
oil_types_directory= 'C:/Users/user/Google Drive/TCD20/python/dataset/Soil types'


predict_df,classification_report1,confusion_matrix1,multi_confusion_matrix1=base_model_predict(grid_search,X_test, y_test)
classification_report_df, dict_matrix, confusion_matrix_df= base_model_summary(grid_search,classification_report1,multi_confusion_matrix1,confusion_matrix1)
classification_report_df.to_csv(os.path.join(Soil_types_directory,'ml_cleaning_image.csv'))

#classification_report_df
print('### Yellow Soil ###')
print(dict_matrix['Yellow Soil'])
print('### Laterite Soil	 ###')
print(dict_matrix['Laterite Soil'])
print('### Black Soil ###')
print(dict_matrix['Black Soil'])
print('### Cinder Soill	 ###')
print(dict_matrix['Cinder Soil'])
print('### Peat Soil	 ###')
print(dict_matrix['Peat Soil'])


print(confusion_matrix_df)
classification_report_df